# Crypto Portfolio Analysis

In [25]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
from binance import Client

In [26]:
# Initialize the Panel Extensions (for Plotly)
import panel as pn
pn.extension("plotly")

# Import panel.interact
from panel.interact import interact

# for merging the dataframes
from functools import reduce

## Instantiate the Binance API Client

In [27]:
# Pulling price data doesn't require an api key as it is publicly available
# https://python-binance.readthedocs.io/en/latest/
# https://binance-docs.github.io/apidocs/spot/en/#compressed-aggregate-trades-list
client = Client()

## Get list of all cryptocurrencies traded on the Binance exchange


In [28]:
exchange_info = client.get_exchange_info()
symbols = exchange_info['symbols']
# Ensure no duplicates by using sets
coin_set = {s['baseAsset'] for s in symbols} | {s['quoteAsset'] for s in symbols}
coin_list = sorted(coin_set)
#coin_list

## Create a DataFrame of the closing prices and dates for each chosen cryptocurrency in the portfolio

In [29]:
def get_historical_data(currency):
    klines = client.get_historical_klines(currency + 'USDT', Client.KLINE_INTERVAL_1DAY, "1 year ago UTC")
    # klines columns=['Open Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close Time', 'Quote asset volume', 'Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore'])
    df = pd.DataFrame(((x[0], x[4]) for x in klines), columns=['timestamp', currency])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)

    return df

# Chosen cryptocurrenies
crypto_portfolio = ('BTC', 'ETH', 'SOL')
# Global dataframe
df = reduce(lambda left,right: pd.merge(left,right,left_index=True, right_index=True, how='outer'), map(get_historical_data, crypto_portfolio))
df.head()

,BTC,ETH,SOL
timestamp,,,
2021-01-11,35404.47000000,1087.01000000,3.10160000
2021-01-12,34051.24000000,1050.16000000,3.63540000
2021-01-13,37371.38000000,1129.89000000,3.52420000
2021-01-14,39144.50000000,1231.62000000,3.29210000
2021-01-15,36742.22000000,1168.53000000,3.24750000


- - -

## Panel Visualisation

In [10]:
# Monte Carlo Simulation Imports
from MCForecastTools import MCSimulation


In [17]:
# Monte Carlo Simulation   

start_date = pd.Timestamp('2021-01-01', tz='America/New_York').isoformat()
end_date = pd.Timestamp('2021-12-31', tz='America/New_York').isoformat()

# Set date as index
df.index = df.index
df.head()

,BTC,ETH,SOL
2021-01-11,35404.47000000,1087.01000000,3.10160000
2021-01-12,34051.24000000,1050.16000000,3.63540000
2021-01-13,37371.38000000,1129.89000000,3.52420000
2021-01-14,39144.50000000,1231.62000000,3.29210000
2021-01-15,36742.22000000,1168.53000000,3.24750000


In [ ]:
# Configuring a Monte Carlo simulation to forecast 10 years cumulative returns

?MCSimulation

MC_traditional_dist = MCSimulation(
    portfolio_data = df,
    weights = [.60,.20,.20],
    num_simulation = 100,
    num_trading_days = 365*10
)


In [ ]:
# Printing the simulation input data
MC_traditional_dist.portfolio_data.head()

In [ ]:
# Plot simulation outcomes

line_plot = MC_traditional_dist.plot_simulation()

In [ ]:
# Plot probability distribution and confidence intervals
dist_plot = MC_traditional_dist.plot_distribution()

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
traditional_tbl = MC_traditional_dist.summarize_cumulative_return()

# Print summary statistics
print(traditional_tbl)

In [ ]:
# Set initial investment
initial_investment = 20000

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $20,000
## rounded to 2 decimal places

ci_lower = round(traditional_tbl[8]*20000,2)
ci_upper = round(traditional_tbl[9]*20000,2)

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 10 years will end within in the range of"
      f" ${ci_lower} and ${ci_upper}")

In [ ]:
# Function for Monte Carlo Simulation

def 
